In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Read 4train data

In [23]:
import pandas as pd
df = pd.read_csv('drug_cleaned.csv')

In [24]:
df.columns

Index(['Unnamed: 0', 'name', 'resolution', 'path', 'has_image', 'im_file'], dtype='object')

In [25]:
df.rename(columns={'Unnamed: 0':'idx'}, inplace=True)

In [26]:
df.columns

Index(['idx', 'name', 'resolution', 'path', 'has_image', 'im_file'], dtype='object')

# Read group data

In [27]:
group = pd.read_csv('group.csv')

In [28]:
group.rename(columns={'Unnamed: 0':'group_idx'}, inplace=True)

In [29]:
group

,group_idx,name
0,0,12 HR Acetazolamide 500 MG Extended Release Or...
1,1,12 HR Acetazolamide 500 MG Extended Release Or...
2,2,12 HR Albuterol 4 MG Extended Release Oral Tablet
3,3,12 HR Amoxicillin 1000 MG / Clavulanate 62.5 M...
4,4,12 HR Bupropion Hydrochloride 100 MG Extended ...
...,...,...
2106,2106,{7 (Ethinyl Estradiol 0.035 MG / norgestimate ...
2107,2107,{7 (Ethinyl Estradiol 0.035 MG / norgestimate ...
2108,2108,{7 (lamotrigine 100 MG Disintegrating Oral Tab...
2109,2109,{84 (Ethinyl Estradiol 0.03 MG / Levonorgestre...


# Merge name and group

In [30]:
df1 = pd.merge(df, group, left_on='name', right_on='name', how='left')

In [34]:
df1

,idx,name,resolution,path,has_image,im_file,group_idx
0,0,Loperamide Hydrochloride 2 MG Oral Capsule,RXNAV 120,image/images/gallery/120/00093-0311-01_RXNAVIM...,True,RXNAV-120_00093-0311-01_RXNAVIMAGE10_26211358.jpg,1052
1,1,Minocycline 50 MG Oral Capsule,RXNAV 120,image/images/gallery/120/00093-3165-01_RXNAVIM...,True,RXNAV-120_00093-3165-01_RXNAVIMAGE10_36231B28.jpg,1137
2,2,Nortriptyline 10 MG Oral Capsule,RXNAV 120,image/images/gallery/120/00093-0810-01_RXNAVIM...,True,RXNAV-120_00093-0810-01_RXNAVIMAGE10_24231228.jpg,1193
3,3,Nortriptyline 25 MG Oral Capsule,RXNAV 120,image/images/gallery/120/00093-0811-01_RXNAVIM...,True,RXNAV-120_00093-0811-01_RXNAVIMAGE10_20231018.jpg,1194
4,4,Nortriptyline 50 MG Oral Capsule,RXNAV 120,image/images/gallery/120/00093-0812-01_RXNAVIM...,True,RXNAV-120_00093-0812-01_RXNAVIMAGE10_2D2316D8.jpg,1195
...,...,...,...,...,...,...,...
47497,51979,Enalapril Maleate 2.5 MG Oral Tablet,RXBASE ORIGINAL,image/images/gallery/original/16714-0442-01_NL...,True,RXBASE-ORIGINAL_16714-0442-01_NLMIMAGE10_DC4D6...,708
47498,51980,quinapril 5 MG Oral Tablet,RXBASE ORIGINAL,image/images/gallery/original/31722-0267-90_NL...,True,RXBASE-ORIGINAL_31722-0267-90_NLMIMAGE10_EA507...,1844
47499,51981,trospium chloride 20 MG Oral Tablet,RXBASE ORIGINAL,image/images/gallery/original/00574-0145-60_NL...,True,RXBASE-ORIGINAL_00574-0145-60_NLMIMAGE10_3C519...,1959
47500,51982,metaxalone 400 MG Oral Tablet,RXBASE ORIGINAL,image/images/gallery/original/64720-0126-10_NL...,True,RXBASE-ORIGINAL_64720-0126-10_NLMIMAGE10_C6506...,1749


# Change class from 0(drug) to group

In [10]:
import os
import shutil
from tqdm.notebook import tqdm
def rmakedir(path):
  if os.path.exists(path):
    shutil.rmtree(path)
    os.makedirs(path)
  else:
    os.makedirs(path)

In [37]:
dst_path = './pseudo_lables'
rmakedir(dst_path)
for i in tqdm(range(len(df1))):
  f, g = df1.im_file.iloc[i], str(df1.group_idx.iloc[i])
#   print(f)
  src = os.path.join('../runs/detect/detect_4train_pseudolabel/labels', f[:-3]+'txt')
  dst = os.path.join(dst_path, f[:-3]+'txt')
  with open(src, 'r') as txt:
      txt = txt.read().splitlines()
      with open(dst, 'w') as dst_file:
          for line in txt:
              cl, x, y, w, h = [i for i in line.split(' ')]
              dst_file.writelines(' '.join([g, x, y, w, h])+'\n')

  0%|          | 0/47502 [00:00<?, ?it/s]

In [39]:
# ทดสอบเปลี่ยน class ในไฟล์ labels
a = 'RXBASE-300_00603-5062-21_NLMIMAGE10_043D820C.txt'
with open('/content/drive/MyDrive/yolov9/runs/detect/detect_4train_pseudolabel/labels/'+a) as txt:
    txt = txt.read().splitlines()
    with open('/content/drive/MyDrive/datasets/drug_c3pi/4train/pseudo_lables/'+a, 'w') as dst_file:
        for line in txt:
            cl, x, y, w, h = [i for i in line.split(' ')]
            dst_file.writelines(' '.join(['abc', x, y, w, h])+'\n')

In [5]:
labels_pseudo = os.listdir('/content/drive/MyDrive/yolov9/runs/detect/detect_4train_pseudolabel/labels')

FileNotFoundError: [WinError 3] The system cannot find the path specified: '/content/drive/MyDrive/yolov9/runs/detect/detect_4train_pseudolabel/labels'

In [4]:
'RXNAV-120_00093-0311-01_RXNAVIMAGE10_26211358.txt' in labels_pseudo

NameError: name 'labels_pseudo' is not defined

In [34]:
for lb in labels_pseudo:
  if 'RX' in lb:
    print(lb)

Streaming output truncated to the last 5000 lines.
RXBASE-120_16729-0023-10_NLMIMAGE10_82464162.txt
RXBASE-120_16729-0022-15_NLMIMAGE10_65413289.txt
RXBASE-120_16729-0031-01_NLMIMAGE10_6440B275.txt
RXBASE-120_16729-0147-17_NLMIMAGE10_D14768EB.txt
RXBASE-120_16729-0021-16_NLMIMAGE10_60413019.txt
RXBASE-120_16729-0020-15_NLMIMAGE10_093F84DC.txt
RXBASE-120_16729-0005-17_NLMIMAGE10_5A12AD75.txt
RXBASE-120_16729-0006-17_NLMIMAGE10_5812AC25.txt
RXBASE-120_16729-0004-17_NLMIMAGE10_913F48EA.txt
RXBASE-120_16714-0683-03_NLMIMAGE10_634E31B1.txt
RXBASE-120_16714-0685-02_NLMIMAGE10_944E4A12.txt
RXBASE-120_16714-0681-02_NLMIMAGE10_DE4EEF57.txt
RXBASE-120_16714-0682-01_NLMIMAGE10_684E3421.txt
RXBASE-120_16729-0007-15_NLMIMAGE10_350E9AF4.txt
RXBASE-120_23155-0116-01_NLMIMAGE10_BD3B5E8A.txt
RXBASE-120_23155-0117-01_NLMIMAGE10_2C461610.txt
RXBASE-120_23155-0110-10_NLMIMAGE10_880E4432.txt
RXBASE-120_23155-0106-10_NLMIMAGE10_F8077C4B.txt
RXBASE-120_23155-0111-01_NLMIMAGE10_4833A44D.txt
RXBASE-120_23155-0